In [ ]:
import pandas as pd
import joblib
import numpy as np
import re
import tldextract
from collections import Counter 
from scipy.stats import entropy 
from urllib.parse import urlparse

In [25]:
predictmodel = joblib.load("main_model.pkl")
print("Model Loaded")

Model Loaded


In [36]:
#function to extract thee features to compare with the given url
def keypoints_that_extract(url_input):
    keypoints = {}
    keypoints["URL lenght"] = len(url_input)
    keypoints["Number of dots"] = url_input.count(".")
    keypoints["Number of slashes"] = url_input.count("/")
    keypoints["Percentage of numerical characters"] = sum(c.isdigit() for c in url_input) / len(url_input)
    def url_entropy():
        frequency = Counter(url_input)
        probability = [frequency[i] / len(url_input) for i in frequency]
        return entropy(probability, base = 2)
    keypoints["Entropy"] = url_entropy()
    
    #for IP address
    IP_address_pattern = r'[0-9]+(?:\.[0-9]+){3}'
    keypoints["IP Address"] = bool(re.search(IP_address_pattern, url_input))
    
    keypoints["Domain name length"] = len(tldextract.extract(url_input).domain)
    keypoints["Repetitions"] = bool(re.search(r'(.)\1{2,}', tldextract.extract(url_input).domain))
    keypoints["Redirections"] = url_input[7:].find("//") != -1
    
    keypoints["Dangerous characters"] = 0  
    keypoints["Dangerous TLD"] = 0  
    keypoints["Suspicious keywords"] = 0  
    keypoints["Entropy and length (PCA)"] = 0 
    
    return pd.DataFrame([keypoints])

In [37]:
url_input = input("Enter the URL: ")

In [38]:
Extracted_keypoints = keypoints_that_extract(url_input)

In [39]:
Extracted_keypoints = Extracted_keypoints[predictmodel.feature_names_in_]

In [40]:
Extracted_keypoints = Extracted_keypoints.astype(float)

In [41]:
predictionmade = predictmodel.predict(Extracted_keypoints)[0] 

In [42]:
#result 
if predictionmade == 1:
    print("The url is safe")
else:
    print("The url is not safe")

The url is not safe
